In [ ]:
# Import dependencies

import gmaps
import pandas as pd
import numpy as np
import requests
import json

# Google developer API key
from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

# Import OpenWeather API key
from config import api_key

In [ ]:
# Read in weather csv
weather_df = pd.read_csv('../WeatherPy/weather.csv')
# # Drop na values
# weather_df.dropna(inplace = True) 
# Drop unnamed column
weather_df.drop(["Unnamed: 0"], axis=1,inplace=True)
weather_df.tail()

In [ ]:
# Store latitude and longitude in locations
locations = weather_df[["latitude","longitude"]]

# Fill NaN values and convert to float
humidity = weather_df["humidity"].astype(float)

In [ ]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=5)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

In [ ]:
# Narrow down weather df based on ideal weather conditions:
# Temp < 30; > 22
# Wind speed < 5
# Cloudiness < 15

ideal_weather_df = weather_df.loc[
    (weather_df["temperature"]>22)
    & (weather_df["temperature"]<30)
    & (weather_df["wind speed"]<5)
    & (weather_df["cloudiness"]<15)
]

ideal_weather_df

In [ ]:
# Set up additional columns to hold information
ideal_weather_df['Hotel Name'] = ""
ideal_weather_df['City'] = ""
ideal_weather_df['Country'] = ""

ideal_weather_df.head()

In [ ]:
# Test retrival of hotel from Google Places API

# Find the closest hotel to coordinates
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "-31.94889169999999,115.8622556",  # The Old Swan Barracks Backpacker Hostel Perth,
    "rankby": "distance",
    "type": "lodging",
    "key": gkey,
}

response = requests.get(base_url, params=params).json()
# print the response URL, avoid doing for public GitHub repos in order to avoid exposing key
# print(response.url)

results = response['results']

print(f"Closest hotel is {results[0]['name']}.")


In [ ]:
# Retrieve all hotels

# Find the closest hotel to coordinates
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "-31.94889169999999,115.8622556",  # The Old Swan Barracks Backpacker Hostel Perth,
    "rankby": "distance",
    "type": "lodging",
    "key": gkey,
}

# Use iterrows to iterate through pandas dataframe
for index, row in ideal_weather_df.iterrows():

    # Get lat and long from df
    latitude = row['latitude']
    longitude = row['longitude']

    # Add keyword to params dict
    params['location'] = f"{latitude},{longitude}"

    # Assemble url and make API request
    print(f"Retrieving Results for Index {index}: {latitude},{longitude}.")
    response = requests.get(base_url, params=params).json()

    # Extract results
    results = response['results']

    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        ideal_weather_df.loc[index, 'Hotel Name'] = results[0]['name']
        ideal_weather_df.loc[index, 'City'] = results[0]['vicinity']
        # ideal_weather_df.loc[index, 'Country'] = results[0]['price_level']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

In [ ]:
# Add column to df for country
ideal_weather_df["Country"] = ""

In [ ]:
# Use Open Weather Map API to retrieve country

base_url = "http://api.openweathermap.org/data/2.5/weather"
params = {
    "q": "albany",
    "appid": api_key,
    "units": "metric"
}

# Use iterrows to iterate through pandas dataframe
for index, row in ideal_weather_df.iterrows():

    # Get city from weather_df
    city = row['city']

    # Add query to params dict
    params['q'] = city

    # Assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}")

    # Extract results
    response = requests.get(base_url, params=params).json()
    
    try:
        ideal_weather_df.loc[index, "Country"] = response["sys"]["country"]
     
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

In [ ]:
# Check df
ideal_weather_df.head()

In [ ]:
# Select coordinates for map markers
coordinates = list(zip(ideal_weather_df.latitude, ideal_weather_df.longitude))
# Assign the marker layer to a variable
markers = gmaps.marker_layer(coordinates)
# Add the layer to the map
fig.add_layer(markers)
fig

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_weather_df.iterrows()]
locations = ideal_weather_df[["latitude", "longitude"]]

In [ ]:
# Add markers, info boxes and heat map
markers = gmaps.marker_layer(locations,info_box_template,
    info_box_content=hotel_info)
fig.add_layer(markers)
fig.add_layer(heat_layer)
fig